# Dashboard Builder: Kepler.gl Interactive Map

This notebook creates an interactive Kepler.gl dashboard with:
1. 500m Walk Isochrone from MARTA stop
2. MARTA transit stops
3. ARC 2020 Census Tracts

**Author**: VIP Team 1270 - Smart Cities / Urban Systems  
**Project**: Atlanta SDG Portfolio Project


In [ ]:
# Imports
import os
import json
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import geopandas as gpd
import requests
from keplergl import KeplerGl

print("Kepler.gl imported successfully")


In [ ]:
# Set up paths
BASE_DIR = os.path.dirname(os.getcwd()) if 'notebooks' in os.getcwd() else os.getcwd()
if 'notebooks' in os.getcwd():
    os.chdir(BASE_DIR)
    
OUTPUT_DIR = os.path.join(BASE_DIR, 'outputs')
DATA_DIR = os.path.join(BASE_DIR, 'data')
DASHBOARD_DIR = os.path.join(BASE_DIR, 'dashboard')
PUBLIC_DASHBOARD_DIR = os.path.join(BASE_DIR, 'public', 'dashboard')

# Create directories
for d in [OUTPUT_DIR, DATA_DIR, DASHBOARD_DIR, PUBLIC_DASHBOARD_DIR]:
    os.makedirs(d, exist_ok=True)

print(f"Base directory: {BASE_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Dashboard directory: {DASHBOARD_DIR}")


## 1. Load Isochrone and MARTA Stops


In [ ]:
# Load isochrone from core_analysis output
isochrone_path = os.path.join(OUTPUT_DIR, 'isochrone_500m.geojson')

if os.path.exists(isochrone_path):
    isochrone_gdf = gpd.read_file(isochrone_path)
    print(f"Loaded isochrone: {len(isochrone_gdf)} features")
    print(f"Columns: {isochrone_gdf.columns.tolist()}")
else:
    print(f"Isochrone file not found at {isochrone_path}")
    print("Please run core_analysis.ipynb first!")
    isochrone_gdf = None


In [ ]:
# Load MARTA stops from core_analysis output
stops_path = os.path.join(OUTPUT_DIR, 'marta_stops_sample.geojson')

if os.path.exists(stops_path):
    stops_gdf = gpd.read_file(stops_path)
    print(f"Loaded MARTA stops: {len(stops_gdf)} features")
else:
    print(f"Stops file not found at {stops_path}")
    print("Please run core_analysis.ipynb first!")
    stops_gdf = None


## 2. Download ARC 2020 Census Tracts


In [ ]:
# ARC 2020 Census Tracts - City of Atlanta
# Using ArcGIS REST API to fetch GeoJSON
# Source: https://opendata.atlantaregional.com/datasets/coaplangis::2020-census-tracts-city-of-atlanta/about

ARC_TRACTS_URL = "https://services1.arcgis.com/Ug5xGQbHsD8zuZzM/arcgis/rest/services/2020_Census_Tracts_City_of_Atlanta/FeatureServer/0/query"

def download_arc_tracts(output_path, force=False):
    """Download ARC 2020 Census Tracts from ArcGIS REST API."""
    if os.path.exists(output_path) and not force:
        print(f"Census tracts already downloaded: {output_path}")
        return gpd.read_file(output_path)
    
    print("Downloading ARC 2020 Census Tracts...")
    
    params = {
        'where': '1=1',  # Get all features
        'outFields': '*',
        'outSR': '4326',  # WGS84
        'f': 'geojson'
    }
    
    try:
        response = requests.get(ARC_TRACTS_URL, params=params, timeout=60)
        response.raise_for_status()
        
        # Save to file
        with open(output_path, 'w') as f:
            f.write(response.text)
        
        print(f"Downloaded and saved to: {output_path}")
        return gpd.read_file(output_path)
        
    except Exception as e:
        print(f"Error downloading census tracts: {e}")
        return None

# Download tracts
tracts_path = os.path.join(DATA_DIR, 'arc_census_tracts_2020.geojson')
tracts_gdf = download_arc_tracts(tracts_path)

if tracts_gdf is not None:
    print(f"Census tracts loaded: {len(tracts_gdf)} features")
    print(f"Columns: {tracts_gdf.columns.tolist()[:10]}...")  # Show first 10 columns


In [ ]:
# Create a simple demo attribute for tract coloring
# (In production, you would fetch ACS median income data)
if tracts_gdf is not None:
    # Add a demo score based on tract area (just for visualization demo)
    tracts_gdf['demo_score'] = (tracts_gdf.geometry.area * 1e6).rank(pct=True) * 100
    tracts_gdf['demo_score'] = tracts_gdf['demo_score'].round(1)
    
    # Add a label
    tracts_gdf['layer_note'] = 'Tract boundaries (demo attribute)'
    
    print("Added demo_score attribute for visualization")
    print(f"Score range: {tracts_gdf['demo_score'].min():.1f} - {tracts_gdf['demo_score'].max():.1f}")


## 3. Create Kepler.gl Map


In [ ]:
# Kepler.gl configuration
kepler_config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': 33.7537,
            'longitude': -84.3901,
            'zoom': 13,
            'bearing': 0,
            'pitch': 0
        },
        'mapStyle': {
            'styleType': 'dark'
        }
    }
}

# Create Kepler.gl map
print("Creating Kepler.gl map...")
map_kepler = KeplerGl(height=600, config=kepler_config)


In [ ]:
# Add layers to map
# Layer 1: Census Tracts (bottom layer)
if tracts_gdf is not None:
    # Select only needed columns to reduce file size
    tracts_export = tracts_gdf[['geometry', 'demo_score', 'layer_note']].copy()
    map_kepler.add_data(data=tracts_export, name='Census Tracts')
    print("Added Census Tracts layer")

# Layer 2: Isochrone polygon
if isochrone_gdf is not None:
    map_kepler.add_data(data=isochrone_gdf, name='500m Walk Isochrone')
    print("Added Isochrone layer")

# Layer 3: MARTA Stops (top layer)
if stops_gdf is not None:
    map_kepler.add_data(data=stops_gdf, name='MARTA Stops')
    print("Added MARTA Stops layer")


In [ ]:
# Display the map (in Jupyter)
map_kepler


## 4. Export Dashboard HTML


In [ ]:
# Export to HTML
dashboard_filename = 'atlanta_dashboard.html'
dashboard_path = os.path.join(DASHBOARD_DIR, dashboard_filename)
public_dashboard_path = os.path.join(PUBLIC_DASHBOARD_DIR, dashboard_filename)

# Save to dashboard directory
map_kepler.save_to_html(file_name=dashboard_path)
print(f"Dashboard saved to: {dashboard_path}")

# Copy to public directory for website embedding
import shutil
shutil.copy(dashboard_path, public_dashboard_path)
print(f"Dashboard copied to: {public_dashboard_path}")


In [ ]:
# Verify export
if os.path.exists(public_dashboard_path):
    size_kb = os.path.getsize(public_dashboard_path) / 1024
    print(f"\n{'='*60}")
    print("DASHBOARD EXPORT COMPLETE")
    print(f"{'='*60}")
    print(f"\nFile: {public_dashboard_path}")
    print(f"Size: {size_kb:.1f} KB")
    print(f"\nThe dashboard is now available at:")
    print(f"  /dashboard/atlanta_dashboard.html")
    print(f"\nView it on the website at: /atlanta-sdg")
else:
    print("Error: Dashboard file was not created")


## Summary

This notebook created an interactive Kepler.gl dashboard with three layers:

1. **Census Tracts**: ARC 2020 Census Tracts for City of Atlanta (demo attribute)
2. **Walk Isochrone**: 500m walk accessibility polygon from sample MARTA stop
3. **MARTA Stops**: Transit stops in downtown Atlanta area

The dashboard is exported to:
- `dashboard/atlanta_dashboard.html` (development copy)
- `public/dashboard/atlanta_dashboard.html` (website-served copy)

**Note**: The tract coloring uses a demo attribute. In production, you would fetch ACS median household income (B19013_001E) via Census API and merge onto tracts.
